# Deep Learning for Text and Sequences

# 6.1. Working with text data
这里是sequence data，这其中text是最广泛的。
可以理解为sequence of characters 或sequene of words，但words层面最常用。

**将text转化为numeric tensors:**
* Each word -> A vector.
* Each character -> A vector
* Each n-grams of words/characters -> A vector.<br>
N-grams是overlapping groups of multiple consecutive words or  characters.

不同分解方式得到的unit称为tokens, 对应的分解称为tokenization.

**两种常用的将vector与token联系起来的方式：**
* One hot encoding
* Token embedding (通常只对words这么做，所以称为word embedding)

**Bag of words**
* 将text划分成set of 2-grams（连续的一个或两个单词组成的gram,可以重叠），这样的set称为bag-of-2-grams.
因为是set,所以没有顺序。
* The family of tokenization methods is called bag-of-words.
* Extracting n-grams 是feature engineering, 在非deep learning，比如logistic regression和random forests中很有用。
* Deep learning中不用这个，我们用hierarchical feature learning.

## One-hot encoding of words and characters
将每个单词用一个独特的整数index表示，再把这个index化为binary vector of size N (N 是vocabulary的size). Vector中只有对应index的位置是1，其它是0.



## Using word embeddings

## Putting it all together: from raw text to word embeddings

## Wrapping up

# 6.2. Understanding recurrent neural networks

## A recurrent layer in Keras

## Understanding the LSTM and GRU layers

## A concrete LSTM example in Keras

## Wrapping up

# 6.3. Advanced use of recurrent neural networks

## A temperature-forecasting problem

## Preparing the data

## A common-sense, non-machine-learning baseline

## A basic machine-learning approach

## A first recurrent baseline

## Using recurrent dropout to fight overfitting

## Stacking recurrent layers

## Using bidirectional RNNs

## Going even further

## Wrapping up

# 6.4. Sequence processing with convnets

## Understanding 1D convolution for sequence data

## 1D pooling for sequence data

## Implementing a 1D convnet

## Combing CNNs and RNNs to process long sequences

## Wrapping up

# Chapter summary